In [ ]:
import pypsa
import pandas as pd
import yaml
import re
import xarray as xr

In [ ]:
def assign_carriers(n):

    if "Load" in n.carriers.index:
        n.carriers = n.carriers.drop("Load")

    if "carrier" not in n.lines:
        n.lines["carrier"] = "AC"

    if n.links.empty:
        n.links["carrier"] = pd.Series(dtype=str)

    config = {
        "AC": {"color": "rosybrown", "nice_name": "HVAC Line"},
        "DC": {"color": "darkseagreen", "nice_name": "HVDC Link"},
    }

    for c in ["AC", "DC"]:
        if c in n.carriers.index:
            continue
        n.carriers = n.carriers.append(pd.Series(config[c], name=c))

def aggregate_costs(n, existing_only=False, by_carrier=True):

    assign_carriers(n)
    
    components = dict(
        Link=("p_nom", "p0"),
        Generator=("p_nom", "p"),
        StorageUnit=("p_nom", "p"),
        Store=("e_nom", "p"),
        Line=("s_nom", None),
    )

    costs = {}
    for c in n.iterate_components(components.keys()):
        p_nom, p_attr = components[c.name]
        if c.df.empty:
            continue
        if not existing_only:
            p_nom += "_opt"
        costs[(c.list_name, "capital")] = (
            (c.df[p_nom] * c.df.capital_cost).groupby(c.df.carrier).sum()
        )
        if p_attr is not None:
            p = c.pnl[p_attr].multiply(n.snapshot_weightings, axis=0).sum()
            if c.name == "StorageUnit":
                p = p.loc[p > 0]
            costs[(c.list_name, "marginal")] = (
                (p * c.df.marginal_cost).groupby(c.df.carrier).sum()
            )
    costs = pd.concat(costs) / 1e9  # bn EUR/a

    if by_carrier:
        costs = costs.groupby(level=2).sum()

    return costs

def get_stats(n):
    stats = pd.concat(
        [
            n.generators.groupby("carrier").p_nom_opt.sum() / 1e3,  # GW
            n.storage_units.groupby("carrier").p_nom_opt.sum() / 1e3,  # GW
            pd.Series(
                {"links": n.links.eval("length * (p_nom_opt) / 1e6").sum()}
            ),  # TWkm
            pd.Series(
                {"lines": n.lines.eval("length * (s_nom_opt) / 1e6").sum()}
            ),  # TWkm
        ]
    )
    
    stats["tsc"] = aggregate_costs(n).sum()

    return stats

In [ ]:
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
# overwrite to subset
config["scenario"]["opts"] = [
    "3H-solar+1.0-onwind+1.5-offwind+1.0-H2+1.0-battery+0.5",
    "3H-solar+1.0-onwind+1.5-offwind+1.5-H2+0.5-battery+1.5",
    "3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5"
]

In [ ]:
scenarios_opts = config["scenario"]["opts"]

In [ ]:
ns = {opts: pypsa.Network(f"networks/elec_s_100_ec_lcopt_{opts}.nc") for opts in scenarios_opts}

In [ ]:
df = pd.concat({opts: get_stats(ns[opts]) for opts in scenarios_opts}, axis=1)

In [ ]:
df

## Building MultiIndex from `opts`

In [ ]:
def parse(opts):
    data = {}
    for o in opts.split("-"):
        s = o.split("+")
        if len(s) > 1:
            carrier = s[0]
            factor = float(s[1])
            data[carrier] = factor
    return pd.Series(data)

In [ ]:
idx_map = pd.concat({opts: parse(opts) for opts in scenarios_opts}, axis=1).fillna(1.)

In [ ]:
df.columns = pd.MultiIndex.from_tuples([idx_map[i] for i in df.columns],
                                       names=[f"{i}-cost" for i in idx_map.index])

In [ ]:
df.to_csv("capacities.csv")

## Convert to `xr.Dataset`

In [ ]:
ds = xr.Dataset.from_dataframe(df.T, sparse=False)

In [ ]:
ds.to_netcdf("capacities.nc")

## Building multiindex from dictionary

params = dict(
    solar=[0.5,1.0],
    onwind=[1.0],
    offwind=[1.0],
    H2=[1.0],
    battery=[1.0])

pd.MultiIndex.from_product(params.values(), names=params.keys())

In [ ]:
n = ns["3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5"]

In [ ]:
[c.split("-")[0] for c in n.carriers.index]